# PLM NSMC Finetuning

## 0. 미션
참조
- model: https://huggingface.co/klue/roberta-base
- data
  - https://github.com/e9t/nsmc
  - https://huggingface.co/datasets/e9t/nsmc

미션
- klue/roberta-base 모델을 활용하여 문장의 감정을 긍정/부정으로 예측하는 감정분류 데이터셋인 NSMC를 fine-tuning을 해 보면서 HuggingFace 사용법 및 PLM 학습 방법을 습득합니다.
- GPU는 6.fine-tuning 과정에서만 사용 하셔도 됩니다.

## 1. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## *2. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 2.1. 라이브러리 Import

In [2]:
# install datasets 라이브러리
!pip install -qq datasets

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from datasets import load_dataset
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TrainingArguments,
                          Trainer)

### 2.2. 환경정보 설정
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.

In [4]:
WORKSPACE = '/content/drive/MyDrive/kt aivle/언어지능 딥러닝/nlp-practice'

In [5]:
!mkdir {WORKSPACE}/data

mkdir: cannot create directory ‘/content/drive/MyDrive/kt’: File exists
mkdir: cannot create directory ‘aivle/언어지능’: No such file or directory
mkdir: cannot create directory ‘딥러닝/nlp-practice/data’: No such file or directory


In [6]:
!ls {WORKSPACE}

ls: cannot access 'aivle/언어지능': No such file or directory
ls: cannot access '딥러닝/nlp-practice': No such file or directory
/content/drive/MyDrive/kt:


## 3. Dataset understanding
- Dataset의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- https://huggingface.co/docs/datasets/quickstart

In [7]:
# Hugging Face Hub에서 dataset 다운로드
# dataset = load_dataset("e9t/nsmc", trust_remote_code=True)
dataset = load_dataset("e9t/nsmc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
# dataset 구조 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [9]:
# train features 확인
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'document': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [10]:
# train num_rows 확인
dataset['train'].num_rows

150000

In [11]:
# train 10개 데이터만 확인
for i  in range(10):
    print(dataset['train'][i])

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
{'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
{'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0}
{'id': '6483659', 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'label': 1}
{'id': '5403919', 'document': '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', 'label': 0}
{'id': '7797314', 'document': '원작의 긴장감을 제대로 살려내지못했다.', 'label': 0}
{'id': '9443947', 'document': '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', 'label': 0}
{'id': '7156791', 'document': '액션이 없는데도 재미 있는 몇안되는 영화', 'label': 1}
{'id': '5912145', 'document': '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', 'label': 1}


### Mission
- Hugging Face Hub 'supark/ko-stsb' dataset을 다운로드 하세요.
- train 데이터의 features, num_rows를 확인하세요.
- train 데이터 값 10개만 출력하세요.

In [12]:
# Hugging Face Hub에서 dataset 다운로드
# dataset = load_dataset('supark/ko-stdb', trust_remote_code=True)
dataset = load_dataset("supark/ko-stsb")

In [13]:
# dataset 구조 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 5709
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 1498
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'score'],
        num_rows: 1378
    })
})

In [14]:
# train features 확인
dataset['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'score': Value(dtype='float64', id=None)}

In [15]:
# train num_rows 확인
dataset['train'].num_rows

5709

In [16]:
# train 10개 데이터만 확인
for i in range(10):
    print(dataset['train'][i])

{'sentence1': '비행기가 이륙하고 있다.', 'sentence2': '비행기가 이륙하고 있다.', 'score': 1.0}
{'sentence1': '한 남자가 큰 플루트를 연주하고 있다.', 'sentence2': '남자가 플루트를 연주하고 있다.', 'score': 0.76}
{'sentence1': '한 남자가 피자에 치즈를 뿌려놓고 있다.', 'sentence2': '한 남자가 구운 피자에 치즈 조각을 뿌려놓고 있다.', 'score': 0.76}
{'sentence1': '세 남자가 체스를 하고 있다.', 'sentence2': '두 남자가 체스를 하고 있다.', 'score': 0.52}
{'sentence1': '한 남자가 첼로를 연주하고 있다.', 'sentence2': '자리에 앉은 남자가 첼로를 연주하고 있다.', 'score': 0.85}
{'sentence1': '몇몇 남자들이 싸우고 있다.', 'sentence2': '두 남자가 싸우고 있다.', 'score': 0.85}
{'sentence1': '남자가 담배를 피우고 있다.', 'sentence2': '남자가 스케이트를 타고 있다.', 'score': 0.1}
{'sentence1': '남자가 피아노를 치고 있다.', 'sentence2': '남자가 기타를 연주하고 있다.', 'score': 0.32}
{'sentence1': '한 남자가 기타를 치고 노래를 부르고 있다.', 'sentence2': '한 여성이 어쿠스틱 기타를 연주하고 노래를 부르고 있다.', 'score': 0.44000000000000006}
{'sentence1': '사람이 고양이를 천장에 던지고 있다.', 'sentence2': '사람이 고양이를 천장에 던진다.', 'score': 1.0}


## 4. Tokenizer understanding
- Tokenizer의 동작 및 사용 방법을 이해하기 위한 과정입니다.
- https://huggingface.co/docs/tokenizers/quicktour

In [17]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
# tokenizer 확인
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [19]:
# 동작 확인을 위한 문장
sentence1 = "지미 카터: 제임스 얼 지미 카터 주니어(1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다."
sentence2 = "수학: 수학(math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다."

In [20]:
# tokenizer tokens 확인
tokens = tokenizer.tokenize(sentence1)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [21]:
# tokens to ids
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18]


In [22]:
# ids to tokens
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [23]:
# [CLS], tokens ids, [SEP] 형식으로 변환
token_ids = tokenizer.encode(sentence1)
print(token_ids)

[0, 19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18, 2]


In [24]:
# 실제 모델에 입력형식으로 변환
# 배열에 미니배치 형식으로 여러 문장을 넣을 수 있음
inputs = tokenizer([sentence1, sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    0, 19537, 20604,    30, 10073,  1411, 19537, 20604, 14577,    12,
         23489,  2440,  3633,  2429,    21,  2210,    97,    13,   793,  4186,
          4327,  3666,  2079,  1545, 24834,  2104,  3698,    12, 13528,    17,
          7561,    13, 30651,    18,     2],
        [    0,  5193,    30,  5193,    12,    80, 16012,    13,  1497,  1295,
            16,  1402,    16,  3962,    16,  4101,    16,  3908,   886,  2079,
          4453,  2069,  5778,  2259,  6204, 28674,    18,     2,     1,     1,
             1,     1,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1

### Mission
- Hugging Face Hub에서 'klue/bert-base' tokenizer를 다운로드 하세요.
- 위 문장을 tokeinze 해 보세요.

In [25]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [26]:
# tokenizer 확인
tokenizer

BertTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [27]:
# tokenizer tokens 확인
tokens = tokenizer.tokenize(sentence1)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [28]:
# tokens to ids
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens_ids)

[19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18]


In [29]:
# ids to tokens
tokens = tokenizer.convert_ids_to_tokens(tokens_ids)
print(tokens)

['지미', '카터', ':', '제임스', '얼', '지미', '카터', '주니어', '(', '1924', '##년', '10', '##월', '1', '##일', '~', ')', '는', '민주당', '출신', '미국', '##의', '제', '##39', '##대', '대통령', '(', '1977', '-', '81', ')', '이다', '.']


In [30]:
# tokens ids 형식으로 변환
tokens_ids = tokenizer.encode(sentence1)
print(tokens_ids)

[2, 19537, 20604, 30, 10073, 1411, 19537, 20604, 14577, 12, 23489, 2440, 3633, 2429, 21, 2210, 97, 13, 793, 4186, 4327, 3666, 2079, 1545, 24834, 2104, 3698, 12, 13528, 17, 7561, 13, 30651, 18, 3]


In [31]:
# 실제 모델에 입력형식으로 변환
# 배열에 미니배치 형식으로 여러 문장을 넣을 수 있음
inputs = tokenizer([sentence1, sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이를 256으로 설정
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    2, 19537, 20604,    30, 10073,  1411, 19537, 20604, 14577,    12,
         23489,  2440,  3633,  2429,    21,  2210,    97,    13,   793,  4186,
          4327,  3666,  2079,  1545, 24834,  2104,  3698,    12, 13528,    17,
          7561,    13, 30651,    18,     3],
        [    2,  5193,    30,  5193,    12,    80, 16012,    13,  1497,  1295,
            16,  1402,    16,  3962,    16,  4101,    16,  3908,   886,  2079,
          4453,  2069,  5778,  2259,  6204, 28674,    18,     3,     0,     0,
             0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1

## 5. Model understanding
- Model의 동작 및 사용 방법을 이해하기 위한 과정입니다.

In [32]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

In [33]:
# Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",
                                                           num_labels=2)  # 예측할 class 개수

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# model 확인
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [35]:
sentence1 = '아 더빙.. 진짜 짜증나네요 목소리'
sentence2 = '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [36]:
# 입력 생성
inputs = tokenizer([sentence1,
                    sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤/ 행렬로 만들어야 하기 때문에 길이를 맞춰줘야함
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    0,  1376,   831,  2604,    18,    18,  4229,  9801,  2075,  2203,
          2182,  4243,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  1963,    18,    18,    18, 11811,  2178,  2088, 28883, 16516,
          2776,    18,    18,    18,    18, 10737,  2156,  2015,  2446,  2232,
          6758,  2118,  1380,  6074,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [37]:
# 추론
# 미세조정(finetuning)을 하지 않았기 때문에 현재는 답변을 신뢰 할 수 없음)
results = model(**inputs)
results

SequenceClassifierOutput(loss=None, logits=tensor([[0.0352, 0.2267],
        [0.0439, 0.2172]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [38]:
# logits 확인
results.logits
# [부정일가능성, 긍정일가능성]

tensor([[0.0352, 0.2267],
        [0.0439, 0.2172]], grad_fn=<AddmmBackward0>)

### Mission
- Hugging Face Hub에서 'klue/bert-base' model 다운로드 하세요.
- 위 문장을 추론해 보세요.

In [39]:
# Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [40]:
# Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base",
                                                           num_labels=2)  # 예측할 class 개수

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# model 확인
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [42]:
sentence1 = '아 더빙.. 진짜 짜증나네요 목소리'
sentence2 = '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [43]:
# 입력 생성
inputs = tokenizer([sentence1,
                    sentence2],
                   padding=True,         # 길이가 짧은 문장에 PAD를 붙여서 길이를 맞춤
                   truncation=True,      # 길이가 너무 긴 문장을 잘라서 버림
                   max_length=256,       # 최대 token 길이
                   return_tensors="pt")  # pytorch 형식으로 값 리턴
inputs

{'input_ids': tensor([[    2,  1376,   831,  2604,    18,    18,  4229,  9801,  2075,  2203,
          2182,  4243,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [    2,  1963,    18,    18,    18, 11811,  2178,  2088, 28883, 16516,
          2776,    18,    18,    18,    18, 10737,  2156,  2015,  2446,  2232,
          6758,  2118,  1380,  6074,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [44]:
# 추론
# 미세조정(finetuning)을 하지 않았기 때문에 현재는 답변을 신뢰 할 수 없음)
results = model(**inputs)
results

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.5628,  0.1741],
        [ 0.3606, -0.3096]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [45]:
# logits 확인
results.logits
# 큰값을 선택하는 것 // ex)아 ~ 긍정이네 or 부정이네

tensor([[ 0.5628,  0.1741],
        [ 0.3606, -0.3096]], grad_fn=<AddmmBackward0>)

## 6. Finetuning
- PLM finetuning 방법을 이해하기 위한 과정입니다.
- TODO 코드를 완성하고 학습을 완료 하세요.

In [46]:
# TODO: Hugging Face Hub에서 dataset 다운로드
dataset = load_dataset("e9t/nsmc")

In [47]:
# TODO: Hugging Face Hub에서 tokenizer 다운로드
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

In [48]:
# TODO: Hugging Face Hub에서 model 다운로드
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base",
                                                           num_labels=2)  # 예측할 class 개수

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
# dataset 선언
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]['document'], self.dataset[idx]['label']

train_dataset = NSMCDataset(dataset['train'])
test_dataset = NSMCDataset(dataset['test'])

In [50]:
# collator 선언
class NSMCCollator:
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch):
        texts, labels = zip(*batch)

        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return_value = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": torch.tensor(labels, dtype=torch.long),
        }

        return return_value

nsmc_collator = NSMCCollator(tokenizer)

In [51]:
# epoch 당 step 수 계산
steps_per_epoch = int(np.ceil(len(dataset['train']) / 128))
steps_per_epoch

1172

In [52]:
# Train arguments 선언
training_args = TrainingArguments(
    output_dir=f"{WORKSPACE}/checkpoint/klue-roberta-base",
    overwrite_output_dir=True,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01, # 오버피팅을 방지하기 위한 값
    lr_scheduler_type='linear', # 0부터 lr 만큼 상승했다가 감소함
    warmup_steps=1000, # lr정점을 찍는 구간이 1000(x축)
    num_train_epochs=3,
    logging_strategy="epoch", #epoch 당 한번씩 수행한다는 의미
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    # bf16=config.fp16,
    # bf16_full_eval=config.fp16,
    fp16=True, # 빠르게 연산하기위함
    fp16_full_eval=True, # 빠르게 연산하기 위함
    half_precision_backend=True,
    load_best_model_at_end=True, # valid가 가장 낮은 상태의 최적의 상태를 로딩해달라라는 의미 train과 겹치는 지점?
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
# trainer 선언
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=nsmc_collator,
    )

In [ ]:
# train
trainer.train()

## 7. Evaluation
- PLM finetuning 후 평가 방법을 이해하기 위한 과정입니다.
- TODO 코드를 완성하고 학습을 완료 하세요.

In [ ]:
# TODO: Hugging Face Hub에서 dataset 다운로드
dataset = load_dataset("e9t/nsmc")

In [ ]:
# 저장된 결과 확인
!ls {WORKSPACE}/checkpoint/klue-roberta-base

In [ ]:
# TODO: 가장 성능이 좋은 경로를 입력하세요.
best_fn = f"{WORKSPACE}/checkpoint/klue-roberta-base/checkpoint-2344"
# 뒷숫자 나주엥 확인해보기

In [ ]:
# TODO: 저장된 tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(best_fn)

In [ ]:
# TODO: 저장된 model 로드
model = AutoModelForSequenceClassification.from_pretrained(best_fn,
                                                          num_labels=2) # 예측할 class 개수

In [ ]:
# dataset 선언
class NSMCDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        super().__init__()

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]['document'], self.dataset[idx]['label']

train_dataset = NSMCDataset(dataset['train'])
test_dataset = NSMCDataset(dataset['test'])

In [ ]:
# collator 선언
class NSMCCollator:
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch):
        texts, labels = zip(*batch)

        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )

        return_value = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": torch.tensor(labels, dtype=torch.long),
        }

        return return_value

nsmc_collator = NSMCCollator(tokenizer)

In [ ]:
# test loader 생성
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    collate_fn=nsmc_collator
)

In [ ]:
# gpu 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
total_correct_cnt = 0
total_sample_cnt = 0

model.to(device)
model.eval()

with torch.no_grad():
    for batch in tqdm(test_loader):
        output = model(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
        )
        probs = torch.functional.F.softmax(output.logits, dim=-1)
        prob, pred = torch.max(probs, dim=-1)

        correct_cnt = (pred.cpu() == batch["labels"]).sum().item()
        sample_cnt = len(batch["labels"])

        total_correct_cnt += correct_cnt
        total_sample_cnt += sample_cnt

print(f"Test Accuracy: {total_correct_cnt / total_sample_cnt * 100:.2f}%")
print(f"Correct / Total: {total_correct_cnt} / {total_sample_cnt}")

In [ ]:
# idx -> label
idx2label = {0: '부정', 1: '긍정'}

In [ ]:
# 랜덤하게 10개 셈플 결과 확인
idxs = np.random.randint(0, dataset['test'].num_rows, 10)

with torch.no_grad():
    for idx in idxs:
        document = dataset['test'][int(idx)]['document']
        # inputs 생성
        inputs = tokenizer(
                    document,
                    truncation=True,
                    max_length=256,
                    return_tensors="pt",
                ).to(device)
        # 추론
        logit = model(**inputs).logits[0]
        prob = F.softmax(logit, dim=-1)
        # 가장 높은 확률을 정답으로 예측
        y = prob.argmax(dim=-1)
        # 출력
        print(f"{idx2label[y.item()]}\t{prob[y].item():.4f}\t{document}")

In [ ]:
# 직접 입력을 받아서 긍정/부정 예측
while True:
    print("input> ", end="")
    document = str(input())
    if len(document) == 0:
        break
    # TODO: inputs 생성
    # TODO: 추론
    # TODO: 가장 높은 확률을 정답으로 예측
    # 출력
    print(f"{idx2label[y.item()]}\t{prob[y].item():.4f}\t{document}")